In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
%matplotlib inline

## Импортируйте данные data.csv.zip
(в папке) с кодировкой ISO-8859-1. Запишите полученный датафрейм в retail, а названия колонок сохраните в переменную retail_columns.

In [3]:
retail = pd.read_csv('data.csv.zip', compression='zip', encoding='ISO-8859-1')

In [4]:
retail_columns = list(retail.columns)


In [5]:
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


## 2. Проверьте, встречаются ли в данных повторяющиеся наблюдения
, и в качестве ответа укажите их количество. Если они есть, то удалите их из retail.


In [8]:
retail.loc[retail.duplicated()] # поиск дубликатов

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


In [7]:
retail = retail.drop_duplicates() # Удаление дубликатов

## 3. Сколько всего заказов отменили пользователи?
Данные содержат в себе записи как и об успешных транзакциях, так и об отмененных. Если пользователь отменил заказ, в начале номера транзакции (InvoiceNo) ставится C (canceled). 



In [11]:
sum_canceled_orders = retail.InvoiceNo.str.startswith('C').sum()
sum_canceled_orders

9251

## 4. Теперь отфильтруйте данные
и оставьте в retail только те заказы, где Quantity > 0. В качестве ответа укажите число оставшихся строк.


In [10]:
retail_quantity =retail.query('Quantity > 1').shape[0]
retail_quantity

381559

## 5. Посчитайте число заказов 
для каждого пользователя (CustomerID) из Германии (Germany). Оставьте только тех, кто совершил более N транзакций (InvoiceNo), где N – 80% процентиль. Запишите полученные id пользователей в germany_top (не весь датафрейм, только id). Идентификатор заказа – InvoiceNo. Для каждого заказа в данных может встречаться более 1 строки.


In [62]:
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [33]:
germany_buy = retail.query('Country =="Germany"') \
    .groupby('CustomerID') \
    .agg({'InvoiceNo': pd.Series.nunique}) \
    .query('InvoiceNo >= InvoiceNo.quantile(q=0.8)') \
    .rename(columns={'InvoiceNo': 'orders_numbers'})


In [34]:
germany_top = germany_buy.index


In [35]:
germany_top

Float64Index([12471.0, 12472.0, 12473.0, 12474.0, 12476.0, 12477.0, 12481.0,
              12500.0, 12569.0, 12600.0, 12619.0, 12621.0, 12626.0, 12647.0,
              12662.0, 12705.0, 12708.0, 12709.0, 12712.0, 12720.0],
             dtype='float64', name='CustomerID')

## 6. Используя объект с id пользователей 
(germany_top), полученный на предыдущем шаге, отфильтруйте наблюдения и оставьте в данных записи только по интересующим нас юзерам. Результирующий датафрейм запишите в top_retail_germany.


In [37]:
top_retail_germany = retail.query('CustomerID in @germany_top')

In [23]:
top_retail_germany

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
12471.0,537381,21807,WHITE CHRISTMAS STAR DECORATION,36,12/6/2010 13:13,0.42,14667.0,United Kingdom
12472.0,537381,22162,HEART GARLAND RUSTIC PADDED,8,12/6/2010 13:13,2.95,14667.0,United Kingdom
12473.0,537381,22700,BLACK AND WHITE DOG BOWL,6,12/6/2010 13:13,2.95,14667.0,United Kingdom
12474.0,537381,85038,6 CHOCOLATE LOVE HEART T-LIGHTS,6,12/6/2010 13:13,2.10,14667.0,United Kingdom
12476.0,537381,22618,COOKING SET RETROSPOT,3,12/6/2010 13:13,9.95,14667.0,United Kingdom
12477.0,537381,47591B,SCOTTIES CHILDRENS APRON,6,12/6/2010 13:13,1.65,14667.0,United Kingdom
12481.0,537382,22728,ALARM CLOCK BAKELIKE PINK,1,12/6/2010 13:13,3.75,16710.0,United Kingdom
12500.0,537382,21644,ASSORTED TUTTI FRUTTI HEART BOX,2,12/6/2010 13:13,1.25,16710.0,United Kingdom
12569.0,537387,79321,CHILLI LIGHTS,4,12/6/2010 13:34,4.95,13468.0,United Kingdom
12600.0,537389,22671,FRENCH LAUNDRY SIGN BLUE METAL,2,12/6/2010 13:41,1.65,17757.0,United Kingdom


## 7.Сгруппируйте top_retail_germany
по коду товара (StockCode). Какой из продуктов добавляли в корзину чаще всего, кроме POST? 
Note: одним заказом считается единовременная покупка любого количества товара, т.е. без учета Quantity.


### вариант 1

In [45]:
top_stock_germany = top_retail_germany \
                    .groupby('StockCode') \
                    .agg({'InvoiceNo': 'count'}) \
                    .sort_values('InvoiceNo', ascending=False)

### вариант 2

In [49]:
top_retail_germany.StockCode.value_counts()

POST     223
22326     64
22423     55
22328     48
22467     38
        ... 
22635      1
22520      1
21054      1
23558      1
22072      1
Name: StockCode, Length: 1259, dtype: int64

## 8. Создайте колонку Revenue 
с суммой покупки, используя колонки Quantity и UnitPrice.  Вернемся к анализу полного датасета retail.


### Вариант 1

In [54]:
retail['Revenue']  = retail.Quantity * retail.UnitPrice

### Вариант 2

In [62]:
retail = retail.assign(Revenue = retail.Quantity * retail.UnitPrice)
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34


## 9. Для каждой транзакции (InvoiceNo), посчитайте финальную сумму заказа.
В качестве ответа укажите топ-5 (InvoiceNo) по сумме заказа (через запятую с пробелом и в порядке убывания TotalRevenue)


In [69]:
retail.query('InvoiceNo == "C581484"')

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
540422,C581484,23843,"PAPER CRAFT , LITTLE BIRDIE",-80995,12/9/2011 9:27,2.08,16446.0,United Kingdom,-168469.6


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France,16.60


In [77]:
top_5 = retail.groupby('InvoiceNo', as_index=False) \
                .agg({'Revenue': 'sum'}) \
                .sort_values('Revenue', ascending=False) \
                .head(5)

In [78]:
top_5

,InvoiceNo,Revenue
22025,581483,168469.60
2303,541431,77183.60
18776,574941,52940.94
19468,576365,50653.91
9741,556444,38970.00


### перевести столбец в строчку

In [79]:
top_5.InvoiceNo.str.cat(sep=', ')

'581483, 541431, 574941, 576365, 556444'